In [1]:
import datetime
from uuid import UUID

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from pydantic_ai import Agent
from pydantic_ai import messages as _messages
from pydantic_ai.models import KnownModelName

from knd.mem_models import Agent as AgentDocument
from knd.mem_models import Memory, Profile, Task, User

%load_ext autoreload
%autoreload 2

In [ ]:
agent = Agent(
    model="google-gla:gemini-1.5-flash",
)



In [2]:
client = AsyncIOMotorClient("mongodb://localhost:27017")

In [3]:
await client.list_database_names()

['admin', 'agent_db', 'config', 'db_name', 'local']

In [4]:
await client.drop_database("agent_db")

In [5]:
await init_beanie(database=client.agent_db, document_models=[User, AgentDocument, Task])

In [7]:
profile = Profile(name="hamza", interests=["football", "python", "ai"])
memories = [
    Memory(
        id=UUID("1ea351e0-5920-47a0-a358-1cc3f1fdda0d"),
        context="last anime episode",
        category="fact",
        content="watched solo leveling season 2 ep 6. was amazing",
    ),
    Memory(context="last ucl match", category="fact", content="barcelona 2-1 real madrid"),
    Memory(
        context="last anime episode",
        category="fact",
        content="watched solo leveling season 2 ep 6. was meh",
        superseded_ids=["1ea351e0-5920-47a0-a358-1cc3f1fdda0d"],
    ),
]

In [8]:
user = User(profile=profile, memories=memories)

In [9]:
# ret_user.memories += [
#     Memory(context="last ucl match", category="fact", content="barcelona 2-1 real madrid"),
#     Memory(
#         context="last anime episode",
#         category="fact",
#         content="watched solo leveling season 2 ep 6. was meh",
#         superseded_ids=["1ea351e0-5920-47a0-a358-1cc3f1fdda0d"],
#     ),
# ]

In [10]:
agent = Agent(
    model="google-gla:gemini-1.5-flash",
    name="joke_teller",
    system_prompt="You are a joke teller. talk like tony stark",
)

In [ ]:
agent_doc = AgentDocument(
    name="joke_teller",
    model="google-gla:gemini-1.5-flash",
    system_prompt="You are a joke teller. talk like tony stark",
)

In [12]:
message_history = [
    _messages.ModelRequest(parts=[_messages.UserPromptPart(content="tell me a joke")]),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Hey there! *adjusts sunglasses* Why don't scientists trust atoms? Because they make up everything! *smirks* Get it? Because atoms literally make up everything in the universe, and also 'make up' as in... ah, you got it. Pure genius, if I do say so myself. JARVIS, add that one to my collection of greatest hits."
            )
        ]
    ),
]


In [13]:
user = await user.insert()
agent_doc = await agent_doc.insert()

In [14]:
task = Task(user=user, agent=agent_doc, message_history=message_history)

In [ ]:
task.message_history

In [ ]:
await task.insert()

In [17]:
task.message_history.append(_messages.ModelRequest(parts=[_messages.UserPromptPart(content="hahahha")]))

In [ ]:
await task.save()

In [25]:
task = await Task.find_one(Task.id == task.id, fetch_links=True)

In [27]:
agent = Agent(model=task.agent.model, name=task.agent.name, system_prompt=task.agent.system_prompt)

In [ ]:
res = await agent.run("more", message_history=task.serialize_message_history(task.message_history))

In [ ]:
res.all_messages()

In [ ]:
task.message_history +=res.all_messages()

In [ ]:
task = await task.save()

In [ ]:
task.message_history

Use [capture_run_messages](https://ai.pydantic.dev/api/agent/#pydantic_ai.agent.capture_run_messages) to access the messages used in a run if it fails.


One way to exit early:
- Use `pydantic-graph`
- Store the `message_history` in the state.
- Check the `message_history` in a `Node` and return `End` if the condition is met.